In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets,transforms

import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [3]:
1+2+3

6

In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(), transforms.Grayscale()
])

In [4]:
train_set = datasets.CIFAR10(
    root="./data/CIFAR10",
    train = True,
    transform=transform,
    download=True
)

test_set = datasets.CIFAR10(
    root="./data/CIFAR10",
    train = False,
    transform = transform,
    download = True
)

train_size = int(len(train_set)*0.9)
val_size = len(train_set)-train_size
train_set,val_set = torch.utils.data.random_split(train_set,[train_size,val_size])

batch_size=128

train_loader = DataLoader(train_set,batch_size,True)
val_loader = DataLoader(val_set,batch_size,True)
test_loader = DataLoader(test_set,batch_size,True)

In [5]:
train_set[0]

(tensor([[[0.2277, 0.2528, 0.2821,  ..., 0.2639, 0.2382, 0.1932],
          [0.4346, 0.4049, 0.3482,  ..., 0.2896, 0.2460, 0.2070],
          [0.3000, 0.2396, 0.1854,  ..., 0.3463, 0.3320, 0.2902],
          ...,
          [0.0591, 0.0441, 0.0437,  ..., 0.3356, 0.2709, 0.2591],
          [0.0450, 0.0594, 0.0423,  ..., 0.3455, 0.2710, 0.2234],
          [0.1256, 0.1155, 0.0857,  ..., 0.2500, 0.2108, 0.2373]]]),
 6)

In [22]:
class simple_cnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,64,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(64,64,kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(64,128,kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(128,128,kernel_size=3,padding=1)
        self.conv5 = nn.Conv2d(128,128,kernel_size=3,padding=1)
        self.conv6 = nn.Conv2d(128,128,kernel_size=3,padding=1)
        self.maxp = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(128*4*4,256)
        self.fc2 = nn.Linear(256,10)
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(128)
        self.bn6 = nn.BatchNorm2d(128)
        self.drop = nn.Dropout(0.5)
    def forward(self,x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.maxp(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.maxp(x)
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.maxp(x)
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x

In [23]:
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs,labels)

            running_loss += loss.item()*images.size(0)

            preds = outputs.argmax(dim=1)
            correct += (preds==labels).sum().item()
    val_loss = running_loss / len(loader.dataset)
    accuracy = correct/len(loader.dataset)

    return val_loss,accuracy

In [24]:
def train_one_epoch(model,optimizer,criterion,loader,device):
    model.train()
    running_loss = 0.0

    for images,labels in loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs=model(images)
        loss = criterion(outputs,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()*images.size(0)
    epoch_loss = running_loss/len(loader.dataset)
    return epoch_loss       

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = simple_cnn().to(device)
print(model)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr = 10e-3,
)

simple_cnn(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(128, eps=1e-05, moment

In [ ]:
num_epochs = 15

train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss = train_one_epoch(
        model=model,loader=train_loader,
        optimizer=optimizer,criterion=criterion,
        device=device
    )
    val_loss,val_acc = evaluate(
        model,val_loader, criterion,device
    )
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"Train Loss: {train_loss:.4f} "
        f"Val Loss: {val_loss:.4f} "
        f"Val Acc: {val_acc:.4f}"
    )  


Epoch [1/15] Train Loss: 2.4451 Val Loss: 2.2391 Val Acc: 0.1638
Epoch [2/15] Train Loss: 2.2422 Val Loss: 2.1416 Val Acc: 0.1754
Epoch [3/15] Train Loss: 2.2212 Val Loss: 2.0448 Val Acc: 0.1932
